In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE LambdaCase #-}

import qualified Data.ByteString as B
import qualified Data.ByteString.Char8 as BC

import Data.List
import Data.Function (on)

readInt bs = case BC.readInt bs of
    Just (i, _) -> i
    _           -> error "invalid int"

adjListEntry pairs = (head (head pairs), map (head . tail) pairs)
toPair [a,b] = (a,b)

contents <- map (toPair . map readInt . BC.words) . BC.lines <$> B.readFile "SCC.txt"

last contents

(875714,542453)

In [21]:
import qualified Data.Graph          as G
import qualified Data.HashMap.Strict as H
import qualified Data.HashSet        as S
import qualified Data.Array          as A

import Prelude hiding (lookup)

import Control.Monad.ST
import Data.STRef
import Control.Monad (forM_, when)
import Data.Maybe (isNothing, fromJust)

import Debug.Trace

tarjan :: G.Graph -> [[Int]]
tarjan graph = runST $ do
    index    <- newSTRef (0 :: Int)
    stack    <- newSTRef []
    stackSet <- newSTRef S.empty
    indices  <- newSTRef H.empty
    lowlinks <- newSTRef H.empty
    output   <- newSTRef []

    forM_ (G.vertices graph) $ \v -> do
        vIndex <- H.lookup v <$> readSTRef indices
        when (isNothing vIndex) $
            strongConnect v graph index stack stackSet indices lowlinks output

    readSTRef output


strongConnect
    :: (Num a, Ord a, Show a)
    => Int
    -> G.Graph
    -> STRef s a
    -> STRef s [Int]
    -> STRef s (S.HashSet Int)
    -> STRef s (H.HashMap Int a)
    -> STRef s (H.HashMap Int a)
    -> STRef s [[Int]]
    -> ST    s ()
strongConnect v graph index stack stackSet indices lowlinks output = do
    i <- readSTRef index
    insert v i indices
    insert v i lowlinks
    increment index
    push stack stackSet v

    forM_ (graph A.! v) $ \w -> lookup w indices >>= \case
        Nothing     -> do
            strongConnect w graph index stack stackSet indices lowlinks output
            vLowLink <- fromJust <$> lookup v lowlinks
            wLowLink <- fromJust <$> lookup w lowlinks
            insert v (min vLowLink wLowLink) lowlinks
        Just wIndex -> do
            wOnStack <- S.member w <$> readSTRef stackSet
            when wOnStack $ do
                vLowLink <- fromJust <$> lookup v lowlinks
                insert v (min vLowLink wIndex) lowlinks

    vLowLink <- fromJust <$> lookup v lowlinks
    vIndex   <- fromJust <$> lookup v indices
    when (vLowLink == vIndex) $ do
        scc <- addSCC v [] stack stackSet
        modifySTRef' output (scc:)
    where
        lookup value hashMap     = H.lookup value <$> readSTRef hashMap
        insert key value hashMap = modifySTRef' hashMap (H.insert key value)
        increment counter        = modifySTRef' counter (+1)

addSCC :: (Eq a, Show a, Num a) => Int -> [Int] -> STRef s [Int] -> STRef s (S.HashSet Int) -> ST s [Int]
addSCC v scc stack stackSet = pop stack stackSet >>= \w -> let scc' = w:scc
    in if w == v then return scc' else addSCC v scc' stack stackSet

push :: Num a => STRef s [Int] -> STRef s (S.HashSet Int) -> Int -> ST s ()
push stack stackSet e = do
    modifySTRef' stack    (e:)
    modifySTRef' stackSet (S.insert e)

pop :: (Eq a, Show a, Num a) => STRef s [Int] -> STRef s (S.HashSet Int) -> ST s Int
pop stack stackSet = do
    e <- head <$> readSTRef stack
    modifySTRef' stack tail
    modifySTRef' stackSet (S.delete e)
    return e

In [22]:
g = G.buildG (1,875714) contents
take 5 $ sortBy (flip compare) . map length $ tarjan g

[434821,968,459,313,211]